In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir kaggle

In [ ]:
!mv kaggle.json kaggle

In [ ]:
!chmod 600 /content/kaggle/kaggle.json

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/kaggle"

In [ ]:
!kaggle datasets download -d ikarus777/best-artworks-of-all-time

In [ ]:
!mkdir artworks && unzip -q best-artworks-of-all-time.zip -d artworks

In [ ]:
import numpy as np
import tensorflow as tf

load_img = tf.keras.preprocessing.image.load_img
img_to_array = tf.keras.preprocessing.image.img_to_array
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

Adam = tf.keras.optimizers.Adam

In [ ]:
import copy
import glob
import pickle as pic
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objs as go
from plotly import tools

In [ ]:
image_path = glob.glob('artworks/images/images/*/*.jpg')
len(image_path)

In [ ]:
vgg16_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

In [ ]:
image_dataset = []

for path in image_path:
    image = load_img(path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image =  tf.keras.applications.vgg16.preprocess_input(image)
    image_dataset.append(image)

In [ ]:
feature_list = []

for image in image_dataset:
    feature = vgg16_model.predict(image)
    feature = feature.flatten()
    feature_list.append(feature)

In [ ]:
def save_feature(filename, feature):
    with open(filename, 'wb') as file:
       pic.dump(feature, file)

In [ ]:
filename = 'feature_list.pkl'
save_feature(filename, feature_list)

In [ ]:
!ls feature_list.pkl -al

In [ ]:
def load_feature(filename):
    with open(filename, 'rb') as file:
        feature = pic.load(file)
        return feature

In [ ]:
feature_list_hdd = load_feature(filename)

In [ ]:
path = 'artworks/images/images/Vincent_van_Gogh/Vincent_van_Gogh_1.jpg'

query_image = load_img(path, target_size=(224, 224))
query_image

In [ ]:
query_image = img_to_array(query_image)
query_image = np.expand_dims(query_image, axis=0)
query_image =  tf.keras.applications.vgg16.preprocess_input(query_image)

query_feature = vgg16_model.predict(query_image)
query_feature = query_feature.flatten()

In [ ]:
nbrs = NearestNeighbors(n_neighbors=10, metric='cosine').fit(feature_list_hdd)

In [ ]:
distances, indices = nbrs.kneighbors([query_feature])

In [ ]:
sub = 0
for i in indices[0]:
    if image_path[i] != path:
        ax = plt.subplot(3, 3, 1 + sub)
        sub+=1
        
        ax.set_xticks([])
        ax.set_yticks([])
        
        result_image = load_img(image_path[i], target_size=(224, 224))
        plt.imshow(result_image)  
        
plt.savefig('result.png', dpi = 300)

In [ ]:
vgg16_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, pooling='avg')

In [ ]:
avg_feature_list = []

for image in image_dataset:
    feature = vgg16_model.predict(image)
    feature = feature.flatten()
    avg_feature_list.append(feature)

In [ ]:
filename = 'avg_feature_list.pkl'
save_feature(filename, avg_feature_list)

In [ ]:
!ls avg_feature_list.pkl -al

In [ ]:
avg_feature_list_hdd = load_feature(filename)

In [ ]:
query_feature = vgg16_model.predict(query_image)
query_feature = query_feature.flatten()

In [ ]:
nbrs = NearestNeighbors(n_neighbors=10, metric='cosine').fit(avg_feature_list_hdd)

In [ ]:
distances, indices = nbrs.kneighbors([query_feature])

In [ ]:
sub = 0
for i in indices[0]:
    if image_path[i] != path:
        ax = plt.subplot(3, 3, 1 + sub)
        sub+=1
        
        ax.set_xticks([])
        ax.set_yticks([])
        
        result_image = load_img(image_path[i], target_size=(224, 224))
        plt.imshow(result_image)  
        
plt.savefig('result.png', dpi = 300)